In [1]:
%%capture
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install --upgrade bitsandbytes
%pip install -U transformers
%pip install --upgrade typing_extensions

In [ ]:
from huggingface_hub import login
import os

hf_token = ""

login(hf_token)

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = "Qwen/Qwen3-8B-Base"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,   
    device_map="auto",  
    torch_dtype=torch.bfloat16,
    trust_remote_code=True             
)

if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
    
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer_config.json:   0%|          | 0.00/9.68k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

...

model-00004-of-00005.safetensors:   0%|          | 0.00/3.19G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [4]:
train_prompt_style = """Below is a fixed instruction that guides the assistant to write a Korean patent specification in full compliance with Korean Patent Law. 
The assistant must use the provided invention details to generate a structured patent section in Korean.
Think step-by-step before responding
The final response should be written in Korean and follow the 2007 model specification guidelines.
**Respond in Korean.**

### Instruction:
당신은 대한민국 특허법에 따라 명세서를 작성하는 특허 어시스턴트입니다. 아래의 구성요소별로 정확하고 구조화된 문서를 생성해야 합니다. 
모든 항목은 특허법 제42조 및 시행규칙 제21조, 개정된 모범 명세서 작성법(2007.07.01. 이후 적용)을 철저히 준수해야 합니다.

1. [발명의 명칭]
- 발명의 내용을 간명하게 표현하는 명칭을 작성합니다.
- 영문명을 {{}} 안에 함께 기재합니다.

2. [기술분야]
- 본 발명이 속하는 기술분야를 간결하게 설명합니다.

3. [배경기술]
- 종래 기술을 서술하고, 가능하면 문헌 인용을 포함합니다.

4. [해결하려는 과제]
- 기존 기술의 문제점과 해결하고자 하는 과제를 기술합니다.

5. [과제의 해결 수단]
- 본 발명이 과제를 어떻게 해결하는지를 구체적으로 기술하고, 청구항과 연결합니다.

6. [발명의 효과]
- 본 발명이 제공하는 구체적이고 비교 가능한 기술적 효과를 기술합니다.

7. [발명을 실시하기 위한 구체적인 내용]
- 실시예, 수단, 수치 등을 포함하여, 평균 기술자가 재현 가능하도록 작성합니다.

8. [도면의 간단한 설명]
- 도면이 있을 경우, 각 도면이 무엇을 나타내는지 기술합니다.

9. [특허청구범위]
- 독립항 및 종속항을 번호와 함께 명확히 구분하여 작성합니다.

※ 모든 기재는 특허법 제42조 제2~5항, 시행규칙 제21조 및 모범명세서 가이드를 기반으로 작성해야 합니다.

### Content:
없음

### Input:
{input}

### Output:
{output}"""


In [5]:
EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

import json
from datasets import Dataset
# ✅ 데이터 로딩
def load_jsonl_dataset(path):
    with open(path, "r", encoding="utf-8") as f:
        data = [json.loads(line) for line in f]
    return Dataset.from_list(data)

# ✅ 포맷팅 함수
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for inp, out in zip(inputs, outputs):
        if not out.endswith(EOS_TOKEN):
            out += EOS_TOKEN
        text = train_prompt_style.format(input=inp, output=out)
        texts.append(text)
    return {"text": texts}

dataset = load_jsonl_dataset("finetuning_data.jsonl")

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
# dataset["text"][10]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [7]:
from peft import LoraConfig, get_peft_model

# LoRA config
peft_config = LoraConfig(
    lora_alpha=16,                           # Scaling factor for LoRA
    lora_dropout=0.05,                       # Add slight dropout for regularization
    r=64,                                    # Rank of the LoRA update matrices
    bias="none",                             # No bias reparameterization
    task_type="CAUSAL_LM",                   # Task type: Causal Language Modeling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Target modules for LoRA
)

model = get_peft_model(model, peft_config)

In [8]:
def filter_too_long(example):
    tokenized = tokenizer(example["text"], truncation=False, return_tensors="pt")
    return tokenized["input_ids"].shape[1] <= 131072

dataset = dataset.filter(filter_too_long)


Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (169994 > 131072). Running this sequence through the model will result in indexing errors


In [9]:
# 1. Dataset 분할
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

# 2. Metrics 함수 (optional)
import math
def compute_metrics(eval_preds):
    loss = eval_preds.loss
    return {
        "eval_loss": loss,
        "perplexity": math.exp(loss) if loss < 100 else float("inf")
    }

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments, EarlyStoppingCallback


# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    eval_strategy="steps",   # ✅ 평가 실행 조건
    eval_steps=50,                 # ✅ 50 스텝마다 평가
    num_train_epochs=3,
    logging_steps=50,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=1e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

Converting train dataset to ChatML:   0%|          | 0/898 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/898 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/898 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/898 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/100 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [11]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

Step,Training Loss,Validation Loss
50,0.833300,0.479861
100,0.465400,0.464089
150,0.462100,0.458186
200,0.448500,0.452831
250,0.454000,0.450795
300,0.445200,0.448292
350,0.429900,0.446281
400,0.430400,0.442894
450,0.432100,0.440909
500,0.414100,0.440521


TrainOutput(global_step=1050, training_loss=0.4354999088105701, metrics={'train_runtime': 1662.9584, 'train_samples_per_second': 1.62, 'train_steps_per_second': 0.81, 'total_flos': 9.99031981473792e+16, 'train_loss': 0.4354999088105701})

In [ ]:
# 이 부분 이상함
# 학습 완료 후 가장 좋은 모델 저장
trainer.save_model("best_model")  # 디렉토리 이름은 원하는 대로

# tokenizer도 함께 저장 (권장)
tokenizer.save_pretrained("best_model")
